# 4. Split data

In [1]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd

In [2]:
ROOT = Path(os.path.abspath('')).resolve().parents[0]
DATA = os.path.join(ROOT, "data")
EXTERNAL_DATA = os.path.join(DATA, "external") 
INTERIM_DATA = os.path.join(DATA, "interim")
RAW_DATA = os.path.join(DATA, "raw")

In [3]:
df = gpd.read_parquet(
    os.path.join(INTERIM_DATA, 'df_feature_engineering.geoparquet.gzip')
)
df

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,...,share_bt_mtb,share_bt_other,attempt_rate_roll7_prev,price_median_clip_roll7_prev,share_bt_diamond_frame_roll7_prev,share_bt_step_through_roll7_prev,share_bt_generic_roll7_prev,share_bt_kids_roll7_prev,share_bt_mtb_roll7_prev,share_bt_other_roll7_prev
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,...,0.0,0.0,1.0,1289.714286,0.357143,0.357143,0.285714,0.000000,0.000000,0.000000
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,...,0.0,0.0,1.0,609.571429,0.214286,0.500000,0.142857,0.142857,0.000000,0.000000
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,...,0.0,1.0,1.0,1916.333333,0.333333,0.166667,0.333333,0.000000,0.000000,0.166667
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,...,0.0,1.0,1.0,1461.500000,0.527778,0.277778,0.055556,0.083333,0.055556,0.000000


In [4]:
val_days  = 60
test_days = 60
gap_days  = 0
date_col  = "date" if "date" in df.columns else "start_date"

## Calculate time borders

In [5]:
d    = pd.to_datetime(df[date_col]).dt.normalize()
last = d.max()
gap  = pd.Timedelta(days=gap_days)

test_start = last - pd.Timedelta(days=test_days) + pd.Timedelta(days=1)
val_end    = test_start - pd.Timedelta(days=1) - gap
val_start  = val_end   - pd.Timedelta(days=val_days) + pd.Timedelta(days=1)
train_end  = val_start - pd.Timedelta(days=1) - gap

## Build masks

In [6]:
m_train = d <= train_end
m_val   = (d >= val_start) & (d <= val_end)
m_test  = d >= test_start

## Split dataset

In [7]:
df_train = df.loc[m_train].copy()
df_val   = df.loc[m_val].copy()
df_test  = df.loc[m_test].copy()

## Save split datasets

In [8]:
df_train.to_parquet(
    os.path.join(INTERIM_DATA, 'df_train.geoparquet.gzip'),
    compression='gzip'
)
df_val.to_parquet(
    os.path.join(INTERIM_DATA, 'df_val.geoparquet.gzip'),
    compression='gzip'
)
df_test.to_parquet(
    os.path.join(INTERIM_DATA, 'df_test.geoparquet.gzip'),
    compression='gzip'
)